Great — I’ll walk you **step-by-step** and give working code files so you can expose your `get_currency_value` tool as a server and call it from your graph or any client.

I’ll provide **two server options** (both small and useful):

* **A. MCP server** — exposes the tool as an MCP tool (WebSocket RPC style). Good if you want other MCP-aware clients to call the tool.
* **B. HTTP mapping server** — a simple FastAPI endpoint that **wraps the same tool** so any HTTP client (requests, browser, LangGraph wrapper) can call it easily.

Then I’ll show:

* How to call the MCP server (async client).
* How to call the HTTP server (sync `requests`).
* How to replace the local `ToolNode` in your graph with a **remote tool wrapper** that calls the HTTP endpoint (so LangGraph uses the servered tool).

---

## Prereqs / install

Run once in your environment:

```bash
pip install mcp langchain langgraph fastapi uvicorn[standard] requests
```

---

## A. MCP server (currency\_mcp\_server.py)

Create `currency_mcp_server.py`:

```python
# currency_mcp_server.py
import asyncio
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("currency-server")

def _get_currency_value(currency_code: str) -> float:
    return {
        "USD": 1.0,
        "INR": 0.012,
        "EUR": 1.09,
    }.get(currency_code.upper(), 0.0)

# Register the tool as an MCP tool (name will be "get_currency_value")
@mcp.tool()
def get_currency_value(currency_code: str) -> dict:
    """Return currency value (value_in_usd) and currency code."""
    value = _get_currency_value(currency_code)
    print(f"[MCP][tool] get_currency_value called: {currency_code} -> {value}")  # debug log
    return {"currency": currency_code.upper(), "value_in_usd": value}

if __name__ == "__main__":
    # This starts the MCP server (blocks the thread)
    print("Starting MCP server (currency-server)...")
    mcp.run()
```

**Run it:**

```bash
python currency_mcp_server.py
```

You should see the MCP server start. The printed lines confirm tool calls.

---

## B. Simple HTTP mapping server (currency\_http.py)

If you prefer plain REST (easier to integrate), create `currency_http.py`:

```python
# currency_http.py
from fastapi import FastAPI, HTTPException, Query
from typing import Optional
import uvicorn

app = FastAPI()

def _get_currency_value(currency_code: str) -> float:
    return {
        "USD": 1.0,
        "INR": 0.012,
        "EUR": 1.09,
    }.get(currency_code.upper(), None)

@app.get("/currency")
def get_currency(code: str = Query(..., description="ISO currency code e.g. INR, USD, EUR"),
                 convert_to: Optional[str] = Query(None, description="Optional convert target, e.g. INR")):
    """
    Returns currency value against USD. If convert_to provided, returns both directions.
    Example: /currency?code=INR  -> {currency, value_in_usd}
             /currency?code=USD&convert_to=INR -> {currency, value_in_usd, 1_usd_in_convert_to}
    """
    val = _get_currency_value(code)
    if val is None:
        raise HTTPException(status_code=404, detail="Currency code not found")

    out = {"currency": code.upper(), "value_in_usd": val}

    # optional reverse conversion: how many target units per USD
    if convert_to:
        convert_val = _get_currency_value(convert_to)
        if convert_val is None or convert_val == 0:
            out["convert_to_rate"] = None
        else:
            # value_in_usd meaning: 1 unit of code = val USD
            # so 1 USD = 1/val units of code.
            # To get 1 USD = X units of convert_to: 1 USD in convert_to = 1/convert_val
            out["1_usd_in_" + convert_to.upper()] = 1.0 / convert_val

    return out

if __name__ == "__main__":
    uvicorn.run("currency_http:app", host="0.0.0.0", port=8000, reload=True)
```

**Run it:**

```bash
uvicorn currency_http:app --reload --port 8000
```

Test in another shell:

```bash
curl "http://localhost:8000/currency?code=INR"
# -> {"currency":"INR","value_in_usd":0.012}

curl "http://localhost:8000/currency?code=USD&convert_to=INR"
# -> {"currency":"USD","value_in_usd":1.0,"1_usd_in_INR":83.3333...}  (depends on your mapping)
```

---

## C. Call the MCP tool from a Python client (async)

Create `client_mcp_call.py`:

```python
# client_mcp_call.py
import asyncio
from mcp.client.session import ClientSession

async def main():
    # connect to running MCP server (default ws://localhost:8000)
    async with ClientSession("ws://localhost:8000") as session:
        # get a reference to the tool
        tool = session.get_tool("get_currency_value")
        # invoke the tool
        res = await tool.invoke({"currency_code": "INR"})
        print("MCP tool result:", res)

if __name__ == "__main__":
    asyncio.run(main())
```

Run:

```bash
python client_mcp_call.py
```

You’ll see the result JSON and the MCP server console prints the debug log.

---

## D. Call the HTTP mapping endpoint (sync, simplest)

```python
import requests

r = requests.get("http://localhost:8000/currency", params={"code": "INR"})
print(r.json())
```

---

## E. Replace your LangGraph tool with remote HTTP tool (so your graph calls server)

If you want your existing LangGraph workflow to call the **HTTP** mapped server instead of the local function, change your tool to a wrapper that calls the HTTP endpoint. Example:

```python
# graph_remote_tool.py
from langchain_core.tools import tool
import requests
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from langchain_core.messages import HumanMessage, AIMessage

class State(TypedDict):
    messages: Annotated[list, add_messages]

@tool
def get_currency_value_remote(currency_code: str) -> float:
    resp = requests.get("http://localhost:8000/currency", params={"code": currency_code}, timeout=5)
    resp.raise_for_status()
    j = resp.json()
    return float(j.get("value_in_usd", 0.0))

# ---- build simple graph that uses remote tool ----
def chatbot_node(state: State):
    # Extract user's last message content
    user_msg = state["messages"][-1]["content"] if isinstance(state["messages"][-1], dict) else state["messages"][-1].content
    # example: if user asks "What is INR in USD?"
    if "INR" in user_msg.upper():
        val = get_currency_value_remote("INR")
        reply = AIMessage(content=f"The value of INR in USD is {val}")
    else:
        reply = AIMessage(content="I can only answer currency queries about INR/USD/EUR in this demo.")
    # append reply
    state["messages"].append(reply)
    return state

builder = StateGraph(State)
builder.add_node(chatbot_node)
builder.add_edge(START, chatbot_node)
builder.add_edge(chatbot_node, END)
graph = builder.compile()

# run
state = graph.invoke({"messages": [{"role": "user", "content": "What is the value of INR in USD?"}]})
print(state["messages"][-1].content)
```

Notes:

* This wrapper calls your HTTP mapping server synchronously; keep a timeout to avoid blocking long.
* You can similarly wrap MCP client call (async) but then you'd need to run graph in async mode.

---

## F. Debugging: How to see tool is called

* For **MCP**: `currency_mcp_server.py` prints `"[MCP][tool] get_currency_value called: ..."` whenever called. Monitor server console logs.
* For **HTTP**: the FastAPI server logs HTTP hits. You can also `print()` inside the `_get_currency_value` function to log calls.
* From LangGraph messages (after `graph.invoke`) examine `state["messages"]` — Agent messages often include action/tool calls (if you use agent tooling).

---

## Notes & tips

* **MCP vs HTTP**:

  * Use **MCP** if you want a standardized RPC tool system and other MCP clients.
  * Use **HTTP** if you want maximum simplicity and integration (any language/client can call it).
* **Security**: don’t hardcode secrets in production. Add auth to the HTTP server (API key) if exposing externally.
* **Port/host**: If you run both MCP and HTTP servers locally, use different ports (MCP default WS port and HTTP 8000).
* **LangGraph async**: If you prefer the graph to call MCP directly (async), your nodes must be async and call `mcp.client.session.ClientSession` — I can give that code if you want it.

---

## Quick checklist to get running now

1. Start MCP server:

   ```bash
   python currency_mcp_server.py
   ```
2. Start HTTP server (in another shell) if you want REST:

   ```bash
   uvicorn currency_http:app --reload --port 8000
   ```
3. Test HTTP:

   ```bash
   curl "http://localhost:8000/currency?code=INR"
   ```
4. Test MCP client:

   ```bash
   python client_mcp_call.py
   ```
5. Use `graph_remote_tool.py` to run LangGraph that calls your HTTP wrapper.

---

If you’d like, I can:

* Provide the **async LangGraph node** that calls the MCP tool (using `ClientSession`) instead of HTTP, or
* Convert your **entire portfolio graph** to call the MCP tool and return totals, or
* Add auth (API key) to the HTTP mapping.

Which one would you like next?
